## NLP Classification 
This is an initiative NLP classification that Mike and I have. We plan to automatically classfify variable names of the output from AMP. 

In [131]:
import pandas as pd
import numpy as np
from textblob.classifiers import NaiveBayesClassifier

## Read and process the data

In [132]:
df = pd.read_csv('C:/Users/zihao.zhang/NLP initiative with Mike/variable classification.csv')
text = df[['Variable', 'Grouping 2']].apply(tuple, axis=1)
text = [i for i in text]
text[1:5]

[('Predicted', 'EXCLUDE'),
 ('Category - Can.Volume Sales - Drug', 'Category'),
 ('Category - Can.Volume Sales - Food', 'Category'),
 ('SNACKS.PR Impressions', 'PR')]

Seperate the sample data to training data and test data

In [133]:
train = text[0:900]
test = text[-30:]

## Model 

Naive Bayesian Classifier. The idea is based on this package: https://textblob.readthedocs.io/en/dev/classifiers.html#classifying-text

In [134]:
cl = NaiveBayesClassifier(train)

In [135]:
cl.accuracy(test)

0.3

The prediction accuracy is actually pretty bad, only 30%. 

We will take a deeper look how's the model works so that we can improve it. The below code shows how this model works. 

Naive Bayesian is simple model, basically it is detecting the important words in each class considering their frequecy in training data. 

So in data process, we need to automatically delete those word with no information. For example, the following shows the model would predict trade when sees 2016. This should be avoid in future modeling.

In [136]:
cl.show_informative_features(10)

Most Informative Features
contains(Diamond.Market) = False           Trade : Interc =     92.3 : 1.0
         contains(Dummy) = False           Trade : Interc =     92.3 : 1.0
          contains(Blue) = False           Trade : Interc =     92.3 : 1.0
         contains(Merch) = True           Compet : Trade  =     74.0 : 1.0
          contains(2016) = True           Hurric : Trade  =     37.7 : 1.0
           contains(Oct) = True           Hurric : Trade  =     37.7 : 1.0
            contains(v2) = True           Compet : Trade  =     24.5 : 1.0
          contains(Week) = True           Distri : Season =     19.5 : 1.0
        contains(BDG-SS) = True           Hurric : Trade  =     16.1 : 1.0
          contains(Bags) = True           Actual : FSIs   =     15.3 : 1.0


Let's take a look at which part the model didn't predict well. The following tuple shows the prediiton. The last column is predicted class while the second column is true class. 

In [137]:
[(test[i][0],test[i][1],cl.classify(test[i][0])) for i in  np.arange(30)]

[('SNACKS.Twitter Costs Variable', 'Social Media', 'Seasonality'),
 ('SNACKS.Facebook Costs Variable', 'Social Media', 'Seasonality'),
 ('BLUE DIAMOND GROWERS-RFG ALM.TPR %ACV - Discount', 'Trade', 'Trade'),
 ('SNACKS.Facebook Costs Variable', 'Social Media', 'Seasonality'),
 ('SNACKS.Instagram Costs Variable', 'Social Media', 'Seasonality'),
 ('SNACKS.Facebook Costs per Imp Variable', 'Social Media', 'Seasonality'),
 ("SNACKS - CANS.Walmart May '16 Rollback", 'Trade', 'Trade'),
 ("SNACKS - BAGS.Walmart April'16 25oz Rollback", 'Trade', 'Trade'),
 ("SNACKS - BAGS.Walmart June '16 14oz Rollback", 'Trade', 'Trade'),
 ('Easter.Dummy', 'Seasonality', 'Trade'),
 ('July 4th.Dummy (WMT)', 'Seasonality', 'Seasonality'),
 ('Memorial Day.Dummy (WMT)', 'Seasonality', 'Seasonality'),
 ('Snacks - CANS.Rollback May-July', 'Trade', 'Seasonality'),
 ('Custom Var Holiday Down - WMT Breeze.2016 Holiday Down',
  'Seasonality',
  'Seasonality'),
 ("ASEPTIC.Rollback Feb-April '16", 'Trade', 'Trade'),
 ('AL

## Future work

In summary, Naive Bayesian is not a good model, it only consider words independently. It is hardly a realy NLP model. But this is a good pipline for us to keep developing more advanced model. 

To be continued 